In [57]:
#################################################################################################################
#
#
#
#  Proyecto: Integración de rasters en un geotiff estandarizado
#  Fecha: 10/06/2019
#  Autores: stu-keil urdaibayc
#  Licencia: 
#  Entrada: archivos .kml con referencia a .tiff
#  Salida: archivos geotiff en formato RGBA
#
#
#
#################################################################################################################

In [4]:
#importa lo que necesitas
import rasterio
from rasterio.merge import merge
import rasterio.plot
import numpy as np
import matplotlib
import rasterio
import os
import pandas as pd
import gdal
from lxml import etree

%matplotlib inline  

In [59]:
###### Manejo de path y extraer nombres de archivos 
os.getcwd()
path_data_raw = "../data/raw/"
path_data_processed = "../data/processed/"
path_data_compo = "../data/cosmeisgay/"
#Remplaza nombres de archivos por _
files = os.listdir(path_data_raw)
[os.rename(str(path_data_raw+files[i]), 
           str(path_data_raw+files[i].replace(" ", "_"))) for i in range(len(files))]
files = []
infilenames = os.listdir("../data/raw/")

In [60]:
#Separar kml de tiffs y nombres de radios
infilenames_kml = [x for x in infilenames if x.find("kml") > 0]
infilenames_tiff =[x for x in infilenames if x.find("tiff") > 0]
radio_names = [x.replace('.kml', '') for x in infilenames_kml]
infilenames_kml.sort()
infilenames_tiff.sort()
radio_names.sort()

In [62]:
for i in range(len(infilenames_kml)):
    if (i % 10) == 0:
        print(i)
    with open(str(path_data_raw+infilenames_kml[i]), 'r') as f:
        tree = etree.parse(f)
    for node in tree.findall('.//kml:name', namespaces={"kml":"http://www.opengis.net/kml/2.2"}):
        node.text = infilenames_tiff[i]
    for node in tree.findall('.//kml:href', namespaces={"kml":"http://www.opengis.net/kml/2.2"}):
        node.text = infilenames_tiff[i]
    with open(str(path_data_raw+infilenames_kml[i]), 'wb') as f:
        tree.write(f)

0
10
20
30
40
50
60
70


In [63]:
for i in range(len(infilenames_tiff)):
    if (i % 10) ==0:
        print(i)
    fn = gdal.Open(str(path_data_raw+infilenames_tiff[i]))
    band = fn.GetRasterBand(1)
    ct   = band.GetRasterColorTable()
    f = open(str(path_data_raw+radio_names[i]+"_rgb_color.txt"), 'w+')    
    for i in range(ct.GetCount()):
        sEntry = ct.GetColorEntry(i)
        alpha = 150
        if (sEntry[0]==sEntry[1])&(sEntry[0]==sEntry[2]):
            alpha = 0  
        f.write( "  %3d. %d %d %d %d\n" % (i, sEntry[0],sEntry[1],sEntry[2],alpha))
    f.close()

0
10
20
30
40
50
60
70


In [64]:
os.chdir(path_data_raw)
path_data_processed = "../processed/"
for i in range(len(infilenames_kml)):
    if (i % 10) == 0:
        print(i)
    os.system(str("gdal_translate -of GTiff " + infilenames_kml[i] + " " +  str(radio_names[i]+"_geo.tif")))
    os.system(str("gdaldem color-relief -of GTiff " + str(radio_names[i]+"_geo.tif") + " "+ str(radio_names[i]+"_rgb_color.txt") +" " +  str(path_data_processed+radio_names[i]+"_geo_color.tif -alpha")))
os.chdir("../../code")
path_data_processed = "../data/processed/"

0
10
20
30
40
50
60
70


In [94]:
files = os.listdir(path_data_processed)
files.sort()
for k in range(len(files)):
    print(str(k)+" - "+files[k])
    ds = gdal.Open(str(path_data_processed+files[k]))
    fn1 = rasterio.open(str(path_data_processed+files[k]))
    ar = ds.ReadAsArray() 
    a = np.zeros([ar.shape[1],ar.shape[2]])
    for i in range(ar.shape[2]):
        #if (i % 500) ==0:
            #print(i)
        for j in range(ar.shape[1]):
            if ar[3,j,i]==0:
                a[j,i] = 0
            elif sum(ar[0:3,j,i]) in range(570,715):
                if ar[0,j,i] in range(200,255):
                    a[j,i] = 105
                elif ar[0,j,i] in range(120,144):
                    a[j,i] = 120
                else:
                    a[j,i] = 135
            elif sum(ar[0:3,j,i]) in range(180,215):
                a[j,i] = 150
            elif sum(ar[0:3,j,i]) in range(245,260):
                if ar[0,j,i] in range(245,260):
                    a[j,i] = 240
                else:
                    a[j,i] = 165
            elif sum(ar[0:3,j,i]) in range(490,510):
                a[j,i] = 195
            elif sum(ar[0:3,j,i]) in range(441,471):
                if ar[0,j,i] in range(245,260):
                    a[j,i] = 210
                else:
                    a[j,i] = 180
            elif sum(ar[0:3,j,i]) in range(400,424):
                a[j,i] = 225
            elif sum(ar[0:3,j,i]) in range(425,440):
                a[j,i] = 255
            else:
                a[j,i] = 0
    out_meta = fn1.meta.copy()
    out_meta.update({"count": 1})
    with rasterio.open(str(path_data_processed+files[k].replace("color","grayscale")), 'w', **out_meta) as dst:
        dst.write(a.astype(rasterio.uint8), 1)

0 - 10_Burkina_Radio_la_voix_du_Paysan_Ouahigouya_geo_color.tif
1 - 11_Burkina_Radio_la_voix_du_Soum_geo_color.tif
2 - 12_Burkina_Radio_lutte_contre_la_desertification_Djibo_geo_color.tif
3 - 13_Cameroon_Radio_Bonne_Semence-Bibemi_geo_color.tif
4 - 14_Cameroon_Radio_Bon_Berger-Kaelé_geo_color.tif
5 - 15_Cameroon_Radio_Demsa_FM-Gaschiga_geo_color.tif
6 - 16_Cameroon_Radio_communautaire_de_Tchatibali-Tchatibali_geo_color.tif
7 - 17_Chad_Radio_Alhouda_Gnere-Gareda_Dar_Tama_geo_color.tif
8 - 18_Chad_Radio_Al-Quouran_Njamena_geo_color.tif
9 - 19_Chad_Radio_Al-hadaba_Karal_geo_color.tif
10 - 1_Burkina_Evangil_Devellopement_Ouagadoudou_geo_color.tif
11 - 20_Chad_Radio_Darbadja__Amtiman_geo_color.tif
12 - 21_Chad_Radio_Dja_FM_Njamena_geo_color.tif
13 - 22_Chad_Radio_Doumchi_Massakory_geo_color.tif
14 - 23_Chad_Radio_Kadaye_Bol_geo_color.tif
15 - 24_Chad_Radio_Mabrouka_Sarh_geo_color.tif
16 - 25_Chad_Radio_Meguedem_Bousso_geo_color.tif
17 - 26_Chad_Radio_Mourgoudida_Massenya_geo_color.tif
18 - 

In [2]:
def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1=np.array(matplotlib.colors.to_rgb(c1))
    c2=np.array(matplotlib.colors.to_rgb(c2))
    return matplotlib.colors.to_hex((1-mix)*c1 + mix*c2)
def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i+int(lv/3)], 16) for i in range(0, lv, int(lv/3)))

In [8]:
c2='#f4f5f5' 
c1='#838ca0' 
n=50
colors = []
[colors.append(colorFader(c1,c2,x/n)) for x in range(n+1)]
colors = [hex_to_rgb(x) for x in colors]

In [ ]:
path_data_processed_grayscale = '../data/processed_grayscale/'
files = os.listdir(path_data_processed_grayscale)
files_grayscale =[x for x in files if x.find("grayscale") > 0]
files_grayscale.sort()

In [ ]:
fn = gdal.Open(str(path_data_processed_grayscale+files_grayscale[0]))
band = fn.GetRasterBand(1)
ar = band.ReadAsArray()
unique = np.unique(ar)
f = open(str(path_data_processed_grayscale+"rgb_color.txt"), 'w+') 
for i in range(len(unique)):
    alpha = 160
    if unique[i] == 0:
        alpha = 0
    sEntry = colors[i]
    f.write( "  %3d. %d %d %d %d\n" % (unique[i], sEntry[0],sEntry[1],sEntry[2],alpha))
f.close()

In [ ]:
os.chdir(path_data_processed_grayscale)
os.system(str("gdaldem color-relief -of GTiff final2.tif rgb_color.txt final2_colorramp.tif -alpha"))
os.chdir("../../code")

In [ ]:
merge_files = []
for i in range(len(files)):
    src = rasterio.open(str(path_data_processed_grayscale+files[i]))
    merge_files.append(src)

In [ ]:
mosaic, out_trans = merge(merge_files)

In [ ]:
out_meta = src.meta.copy()
out_meta.update({"height": mosaic.shape[1],"width": mosaic.shape[2],"transform": out_trans})
with rasterio.open(str(path_data_compo+"mosaic1.tif"), "w", **out_meta) as dest:
    dest.write(mosaic)

In [ ]:
#rasterio.plot.show(mosaic)